In [1]:
import { TextLoader } from "langchain/document_loaders/fs/text";
const loader = new TextLoader("data/qiu.txt")
const docs = await loader.load();
docs

[
  Document {
    pageContent: "三体前传：球状闪电 作者：刘慈欣\n" +
      "\n" +
      "内容简介：\n" +
      "　　没有《球状闪电》，就没有后来的《三体》！\n" +
      "　　《三体》前传！\n" +
      "　　亚洲首位雨果奖得主刘慈欣的三大长篇之一！（《三体》《球状闪电》《超新星纪元》）\n" +
      "　"... 189395 more characters,
    metadata: { source: "data/qiu.txt" }
  }
]

In [ ]:
import { RecursiveCharacterTextSplitter } from 'langchain/text_splitter';
import { OllamaEmbeddings } from "@langchain/ollama";
import { MemoryVectorStore } from "langchain/vectorstores/memory";


const splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 500,
    chunkOverlap: 100,
});
const splitDocs = await splitter.splitDocuments(docs);
// console.log(splitDocs[4])
// console.log(splitDocs[5].pageContent)
const embeddings = new OllamaEmbeddings({
    model: "llama3",
    baseUrl: "http://127.0.0.1:11434"
}
);
const vectorstore = new MemoryVectorStore(embeddings);
try {
    await vectorstore.addDocuments(splitDocs);
    console.log("Documents added successfully:", vectorstore.memoryVectors);
} catch (error) {
    console.error("Error adding documents:", error);
}

const retriever = vectorstore.asRetriever(2)
// retriever
const res = await retriever.invoke("原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论")
// res

: 

In [ ]:
const convertDocsToString = (documents: Document[]): string => {
  return documents.map((doc) => doc.pageContent).join("\n");
};

: 

In [ ]:
const contextRetriverChain = RunnableSequence.from([
    (input) => input.question,
    retriever,
    convertDocsToString
])

: 

In [ ]:
const result = await contextRetriverChain.invoke({ question: "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论" })
console.log(result)

: 

In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts"
const TEMPLATE = `
你是一个熟读刘慈欣的《球状闪电》的终极原著党，精通根据作品原文详细解释和回答问题，你在回答时会引用作品原文。
并且回答时仅根据原文，尽可能回答用户问题，如果原文中没有相关内容，你可以回答“原文中没有相关内容”，

以下是原文中跟用户回答相关的内容：
{context}

现在，你需要基于原文，回答以下问题：
{question}`;

const prompt = ChatPromptTemplate.fromTemplate(TEMPLATE);

: 

In [ ]:
import { Ollama } from "@langchain/community/llms/ollama";
const model = new Ollama();

: 

In [ ]:
const ragChain = RunnableSequence.from([
   {
    context: contextRetriverChain,
    question: (input) => input.question,
   },
   prompt,
   model,
   new StringOutputParser(),
])

const answer = await ragChain.invoke({
   quention: "详细描述原文中有什么跟直升机相关的场景"
})
console.log(answer)

: 